## Recommendation System
Collaborative filtering with implicit feedback based on latent factors. Prepare data on user-item relationships for each user-company in format that ALS can use.
We require each unique assignee ID in the rows of the matrix, and each unique item ID in columns of matrix.
Values of matrix should be (?) binary user-item preference * confidence

In [1]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import pandas as pd
import numpy as np

from test_model import (get_patent_fields_list, get_ml_patents, 
                        create_title_abstract_col,trim_data, 
                        structure_dataframe, partition_dataframe, 
                        build_pipeline, process_docs, pat_inv_map, get_topics)

from rec_system import alphanum_to_int, int_to_alphanum, get_pat_recs

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary, mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel
from gensim.test.utils import common_dictionary, datapath, temporary_file
from smart_open import smart_open

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re
import os
import calendar
import requests
from bs4 import BeautifulSoup
import pickle
import math

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

%load_ext autoreload
%autoreload 2

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark

In [3]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

### Data understanding - Acquire data

#### Data understanding - Acquire data for text workflows

In [4]:
# load pickled dataset
with open('/Users/lee/Documents/techniche/techniche/data/raw_data_1000', 'rb') as f:
    raw_data_1000 = pickle.load(f)

In [5]:
# define keys as criteria to subset dataset #1 for non-text workflows
retained_keys = ['patent_number', 'patent_firstnamed_assignee_id']

# subset raw dataset by desired keys/columns
data_1000 = trim_data(data=raw_data_1000, keys=retained_keys)

In [6]:
# define keys as criteria to subset dataset #2, for text workflows
retained_keys_2 = ['patent_number', 'patent_firstnamed_assignee_id',
                   'patent_title', 'patent_abstract']

# subset raw dataset by desired keys/columns for text analysis workflows
data_1000_2 = trim_data(data=raw_data_1000, keys=retained_keys_2)

#### Data preparation

In [7]:
# create new item in dataset #2 by concat of patent_title and patent_abstract
data_1000_2 = create_title_abstract_col(data=data_1000_2)

In [8]:
# create Pandas dataframe from dataset #1
df_1000 = pd.DataFrame(data_1000)

In [9]:
# create Pandas dataframe from dataset #2
df_1000_2 = pd.DataFrame(data_1000_2)
df_1000_2.head(3)

,patent_abstract,patent_firstnamed_assignee_id,patent_number,patent_title,patent_title_abstract
0,"Tools and techniques for the rapid, continuous...",org_VU2IXnxgxGIK8A8oQrwm,10226194,"Statistical, noninvasive measurement of a pati...","Statistical, noninvasive measurement of a pati..."
1,The disclosure relates to structural health mo...,org_9cmRc2rH8nbl8O9VuxYL,10228278,Determining a health condition of a structure,Determining a health condition of a structure....
2,A scenario is defined that including models of...,org_8O8xQifxyiW5pZB2KuDx,10228693,Generating simulated sensor data for training ...,Generating simulated sensor data for training ...


In [10]:
# for dataset #1: drop row that contains invalid data
df_1000[df_1000.patent_number.str.contains('[RE]')]
df_1000 = df_1000.drop(df_1000.index[[717]])

# drop NaNs in patent_firstnamed_assignee_id column
df_1000 = df_1000.dropna()

In [11]:
# for dataset#2: drop row that contains invalid data
df_1000_2[df_1000_2.patent_number.str.contains('[RE]')]
df_1000_2 = df_1000_2.drop(df_1000_2.index[[717]])

# drop NaNs in patent_firstnamed_assignee_id column
df_1000_2 = df_1000_2.dropna()

#### Data preparation - model #1
Prepare data on user-item relationships for each user-company in format that ALS can use.
We require each unique assignee ID in the rows of the matrix, and each unique item ID in columns of matrix.
Values of matrix should be (?) binary user-item preference * confidence

In [12]:
# create new rating column and assign value of 1
df_1000['rating'] = 1

In [13]:
# convert patent_number column from string to int
df_1000 = df_1000.astype({'patent_number': 'int64'})
# uncomment to confirm
# df_1000.info()

In [14]:
# convert alphanumeric patent_firstnamed_assignee_id col to int
df_1000 = df_1000.astype({'patent_number': 'int64'})

In [15]:
# df_1000['patent_firstnamed_assignee_id'] = df_1000['patent_firstnamed_assignee_id'].apply(hash).apply(abs)
df_1000['patent_firstnamed_assignee_id'] = df_1000['patent_firstnamed_assignee_id'].apply(hash).apply(abs) % 65536 # 2^16

In [16]:
# df_1000['patent_firstnamed_assignee_id'] = df_1000['patent_firstnamed_assignee_id'].apply(hash).apply(abs)
df_1000['patent_number'] = df_1000['patent_number'] % 65536 # 2^16

In [17]:
df_1000 = df_1000.astype({'patent_firstnamed_assignee_id': 'int'})

#### Data preparation - model #1 - create Spark dataframe from pandas dataframe

In [18]:
sp_df_1000 = spark.createDataFrame(df_1000)

In [19]:
# cast columns from bigint to int
sp_df_1000_2 = sp_df_1000.withColumn("patent_firstnamed_assignee_id",
                                     sp_df_1000["patent_firstnamed_assignee_id"]
                                     .cast(IntegerType())).withColumn("patent_number",
                                     sp_df_1000["patent_number"]
                                     .cast(IntegerType())).withColumn("rating", 
                                     sp_df_1000["rating"].cast(IntegerType()))

In [20]:
# partition dataframe 
(training, test) = sp_df_1000.randomSplit([0.8, 0.2])

### Model # 1
Build the recommendation model using ALS on the training data

In [21]:
# build ALS recommendation model
als = ALS(maxIter=5,
          regParam=0.01, 
          rank=10, # number of latent topics- ME-10?
          alpha=30,
          implicitPrefs=True, # # implicitPrefs=True b/c ratings are implicit
          userCol="patent_firstnamed_assignee_id", 
          itemCol="patent_number", 
          ratingCol="rating",
          coldStartStrategy="nan") # coldStartStrategy="nan" to retain NaNs

In [22]:
# fit ALS model to the training set
model = als.fit(training)

#### Model #1 - Evaluation - Compare to naive baseline
Compare model evaluation result with naive baseline model that only outputs (for explicit - the average rating (or you may try one that outputs the average rating per movie).

#### Model #1 - Optimize model

In [23]:
# optimize model

#### Getting Predictions

In [24]:
# get predictions for test set
predictions_test = model.transform(test)
predictions_test_df = predictions_test.toPandas()

In [25]:
# get predictions for training set
predictions_train = model.transform(training)
predictions_train_df = predictions_train.toPandas()
predictions_train_df

,patent_firstnamed_assignee_id,patent_number,rating,prediction
0,743,3997,1,0.978062
1,49710,9900,1,0.358418
2,743,26087,1,0.978062
3,32880,43256,1,0.358176
4,21785,54258,1,0.873526
5,49407,34602,1,0.357778
6,65206,2525,1,0.885076
7,53148,3986,1,0.358143
8,4219,4042,1,0.358017
9,31696,27977,1,0.643098


In [26]:
predictions_train_df.dropna()

,patent_firstnamed_assignee_id,patent_number,rating,prediction
0,743,3997,1,0.978062
1,49710,9900,1,0.358418
2,743,26087,1,0.978062
3,32880,43256,1,0.358176
4,21785,54258,1,0.873526
5,49407,34602,1,0.357778
6,65206,2525,1,0.885076
7,53148,3986,1,0.358143
8,4219,4042,1,0.358017
9,31696,27977,1,0.643098


In [27]:
predictions_test_df.dropna()

,patent_firstnamed_assignee_id,patent_number,rating,prediction


### Model #2 - Data preparation

In [28]:
train_text, test_text = train_test_split(df_1000_2, test_size = 0.2)

#### Model 2 - Data preparation - text data

- TF-IDF vectorization of patents - metrics - avg distance between individual patents, with ranking
- take tf-idf vector and argsort by absolute value, to see which features are most important to patent
- get top 20 features. normally would do cosine distance betweel all vectors. BUT, only do cosine distance between these top 20 features, for cold start patents

In [29]:
# instantiate TF-IDF Vectorizer using standard English stopwords
tfidf = TfidfVectorizer(stop_words='english')

In [30]:
# fit TF-IDF matrix on text column
tfidf_matrix = tfidf.fit_transform(train_text['patent_title_abstract'])

In [31]:
# output matrix, 972 docs, 5364 terms
tfidf_matrix.shape

(777, 4884)

### Model 3 - compute distance metric

In [32]:
# compute cosine similarity matrix between docs using linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [33]:
# construct reverse map of indices and pat_title_abstract
indices = pd.Series(train_text.index, index = train_text['patent_number']).drop_duplicates()

In [34]:
# tfidf vec requires list, not just string
unseen_data = test_text
unseen_data

,patent_abstract,patent_firstnamed_assignee_id,patent_number,patent_title,patent_title_abstract
290,A distributed processing system is disclosed h...,org_zlS8am8xXIliuhSmrhev,10148589,Resource allocation in distributed processing ...,Resource allocation in distributed processing ...
80,A natural language question answering method a...,org_myRnscKfY7JOy5h8LVrg,10210245,Natural language question answering method and...,Natural language question answering method and...
515,"A method, apparatus and computer program produ...",org_QCpO1mfVxBoSVUCIF5PT,10074198,Methods and apparatuses for image processing a...,Methods and apparatuses for image processing a...
332,A computer system identifies malicious Uniform...,org_rGrTOVRIsnln8BjhEMKM,10135863,Malicious software detection in a computing sy...,Malicious software detection in a computing sy...
393,A mechanism is described for facilitating reco...,org_BhFWbZ5cX0tSnPE1cE4T,10108850,"Recognition, reidentification and security enh...","Recognition, reidentification and security enh..."
571,A user initializes multi-factor authentication...,org_Vbc6obpnxWM42d0HjlXY,10057227,Determination of authentication mechanism,Determination of authentication mechanism. A u...
686,Techniques are described for analyzing communi...,org_tQZg8KwP0X8Per8Nk6Ho,10015316,Response quality index for service sessions,Response quality index for service sessions. T...
489,The disclosed solution uses machine learning-b...,org_m0hwKu6AOhMMpHWb6CrO,10078689,Labeling/naming of themes,Labeling/naming of themes. The disclosed solut...
899,"A system for processing a report, comprising a...",org_ICUTrOk0VehJ4mFMbCqQ,9922026,System and method for processing a natural lan...,System and method for processing a natural lan...
384,A security platform employs a variety techniqu...,org_FnwHVjRQWWObkacCtbJp,10116670,Event specific relationship graph generation a...,Event specific relationship graph generation a...


In [35]:
unseen_tfidf = tfidf.transform(unseen_data['patent_title_abstract'])
unseen_tfidf.shape

(195, 4884)

In [36]:
# pass patent number from training set to get_recommendations
# take user input of string and output most similar documents
get_pat_recs('10019674')

NameError: name 'matrix' is not defined

In [ ]:
train_text.head()

#### Model #2 - Apply K means clustering to distance matrix

In [ ]:
km = KMeans(20)

In [ ]:
kmresult = km.fit(tfidf_matrix).predict(unseen_tfidf)

In [ ]:
kmresult_p = km.predict(unseen_tfidf)

In [ ]:
kmresult_p